In [1]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from descartes import PolygonPatch

In [4]:
df = pd.read_csv("data/full_run_4.zip",skiprows=6)

In [5]:
df

,[run number],size-of-std,transform-houses?,avg_migrant_income,amount-of-health-improved,aggregate-cluster-size,improve-health?,amount-of-mixed-nhs,citizen-color,build-percent,...,KPI-p-sc-upper,KPI-p-dutch,KPI-p-other-western,KPI-p-antilles,KPI-p-morocco,KPI-p-suriname,KPI-p-turkey,KPI-p-indonesian,KPI-p-eastern-eu,KPI-p-other-nonwestern
0,15,0.125,False,16000,5,1,False,4,social group,2.5,...,0,0,0,0,0,0,0,0,0,0
1,25,0.125,False,16000,5,1,False,4,social group,2.5,...,0,0,0,0,0,0,0,0,0,0
2,15,0.125,False,16000,5,1,False,4,social group,2.5,...,[[105 0.052] [98 0] [73 0.688] [8 0.039] [113 ...,[[105 0.515] [98 0.267] [73 0.614] [8 0.702] [...,[[105 0.06] [98 0.035] [73 0.178] [8 0.13] [11...,[[105 0.015] [98 0.05] [73 0.006] [8 0.008] [1...,[[105 0.05] [98 0.14] [73 0] [8 0.011] [113 0....,[[105 0.029] [98 0.023] [73 0.076] [8 0.027] [...,[[105 0.129] [98 0.113] [73 0.009] [8 0.021] [...,[[105 0.074] [98 0.166] [73 0] [8 0.007] [113 ...,[[105 0.012] [98 0.066] [73 0.038] [8 0.024] [...,[[105 0.116] [98 0.14] [73 0.079] [8 0.07] [11...
3,3,0.125,False,16000,5,1,False,4,social group,2.5,...,0,0,0,0,0,0,0,0,0,0
4,8,0.125,False,16000,5,1,False,4,social group,2.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11803,284,0.125,False,22000,5,1,False,4,social group,2.5,...,[[46 0.699] [116 0] [51 0.158] [92 0.072] [115...,[[46 0.564] [116 0] [51 0.732] [92 0.913] [115...,[[46 0.221] [116 0] [51 0.085] [92 0.026] [115...,[[46 0.008] [116 0] [51 0.009] [92 0.002] [115...,[[46 0.006] [116 0] [51 0.003] [92 0] [115 0.0...,[[46 0.048] [116 0] [51 0.047] [92 0.028] [115...,[[46 0.009] [116 0] [51 0.029] [92 0.004] [115...,[[46 0.009] [116 0] [51 0.006] [92 0.002] [115...,[[46 0.029] [116 0] [51 0.031] [92 0.01] [115 ...,[[46 0.106] [116 0] [51 0.058] [92 0.015] [115...
11804,288,0.125,False,22000,5,1,False,4,social group,2.5,...,[[99 0.372] [92 0.047] [48 0.472] [10 0.006] [...,[[99 0.638] [92 0.937] [48 0.504] [10 0.346] [...,[[99 0.117] [92 0.026] [48 0.274] [10 0.079] [...,[[99 0.01] [92 0.002] [48 0.003] [10 0.041] [4...,[[99 0.015] [92 0] [48 0] [10 0.032] [41 0.002...,[[99 0.046] [92 0.014] [48 0.073] [10 0.022] [...,[[99 0.056] [92 0.003] [48 0.015] [10 0.076] [...,[[99 0.02] [92 0] [48 0.009] [10 0.051] [41 0]...,[[99 0.046] [92 0.005] [48 0.035] [10 0.079] [...,[[99 0.052] [92 0.013] [48 0.087] [10 0.274] [...
11805,286,0.125,False,22000,5,1,False,4,social group,2.5,...,[[114 0.081] [15 0] [7 0.707] [110 0] [48 0.55...,[[114 0.533] [15 0.007] [7 0.561] [110 0] [48 ...,[[114 0.065] [15 0.003] [7 0.269] [110 0] [48 ...,[[114 0.021] [15 0.003] [7 0.008] [110 0] [48 ...,[[114 0.03] [15 0.01] [7 0.002] [110 0] [48 0....,[[114 0.033] [15 0] [7 0.044] [110 0] [48 0.05...,[[114 0.119] [15 0.009] [7 0.015] [110 0] [48 ...,[[114 0.057] [15 0.015] [7 0.003] [110 0] [48 ...,[[114 0.027] [15 0.007] [7 0.03] [110 0] [48 0...,[[114 0.115] [15 0.946] [7 0.068] [110 1] [48 ...
11806,288,0.125,False,22000,5,1,False,4,social group,2.5,...,[[68 0.85] [37 0] [116 0] [93 0.762] [105 0.06...,[[68 0.56] [37 0.007] [116 0] [93 0.618] [105 ...,[[68 0.16] [37 0.003] [116 0] [93 0.121] [105 ...,[[68 0] [37 0.003] [116 0] [93 0.018] [105 0.0...,[[68 0] [37 0.004] [116 0] [93 0.014] [105 0.0...,[[68 0.08] [37 0.001] [116 0] [93 0.056] [105 ...,[[68 0] [37 0.006] [116 0] [93 0.047] [105 0.1...,[[68 0] [37 0.007] [116 0] [93 0.016] [105 0.1...,[[68 0] [37 0.012] [116 0] [93 0.031] [105 0.0...,[[68 0.2] [37 0.957] [116 0] [93 0.079] [105 0...
